1. Try a Support Vector Machine regressor (sklearn.svm.SVR), with various hyperparameters such as kernel="linear" (with various values for the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Don’t worry about what these hyperparameters mean for now. How does the best SVR predictor perform?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv",header=0)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

As the total_bedrooms got 207 missing value, so we need to impute the missing value or drop value

In [ ]:
df["rooms_per_household"] = df["total_rooms"]/df["households"]
df["bedrooms_per_room"] = df["total_bedrooms"]/df["total_rooms"]
df["population_per_household"]=df["population"]/df["households"]

In [ ]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467


In [ ]:
num=df.drop(['ocean_proximity','median_house_value'],axis=1)
cat=df[['ocean_proximity']]
label=df[['median_house_value']]

In [ ]:
impute=SimpleImputer(strategy='median')

In [ ]:
num_pipline=Pipeline([('impute',impute),('standard_scale',StandardScaler())])
num_features=num_pipline.fit_transform(num)
num_df=pd.DataFrame(num_features,columns=num.columns)
num_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
0,-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.344766,0.628559,-1.149930,-0.049597
1,-1.322844,1.043185,-0.607019,2.045890,1.357143,0.861439,1.669961,2.332238,0.327041,-0.990381,-0.092512
2,-1.332827,1.038503,1.856182,-0.535746,-0.827024,-0.820777,-0.843637,1.782699,1.155620,-1.445865,-0.025843
3,-1.337818,1.038503,1.856182,-0.624215,-0.719723,-0.766028,-0.733781,0.932968,0.156966,-0.493627,-0.050329
4,-1.337818,1.038503,1.856182,-0.462404,-0.612423,-0.759847,-0.629157,-0.012881,0.344711,-0.707889,-0.085616


After we fill in the missing value with the median of the value, we can do the feature scaling
for independent variables, we can do the Standard scaling
for category variable, we can do the onehot encoding but if it got too many category, we need to perform robust encoding instead, lets see how many categories we got

In [ ]:
cat['ocean_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [ ]:
cat.head()

,ocean_proximity
0,NEAR BAY
1,NEAR BAY
2,NEAR BAY
3,NEAR BAY
4,NEAR BAY


In [ ]:
OHE=OneHotEncoder(sparse=False)
cat_feature=OHE.fit_transform(cat)
cat_feature

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [ ]:
# showing the categories
OHE.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [ ]:
cat_df=pd.DataFrame(cat_feature,columns=OHE.categories_)
cat_df.head()

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0


In [ ]:
X=pd.concat([num_df,cat_df],axis=1)
X.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household,"(<1H OCEAN,)","(INLAND,)","(ISLAND,)","(NEAR BAY,)","(NEAR OCEAN,)"
0,-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.344766,0.628559,-1.149930,-0.049597,0.0,0.0,0.0,1.0,0.0
1,-1.322844,1.043185,-0.607019,2.045890,1.357143,0.861439,1.669961,2.332238,0.327041,-0.990381,-0.092512,0.0,0.0,0.0,1.0,0.0
2,-1.332827,1.038503,1.856182,-0.535746,-0.827024,-0.820777,-0.843637,1.782699,1.155620,-1.445865,-0.025843,0.0,0.0,0.0,1.0,0.0
3,-1.337818,1.038503,1.856182,-0.624215,-0.719723,-0.766028,-0.733781,0.932968,0.156966,-0.493627,-0.050329,0.0,0.0,0.0,1.0,0.0
4,-1.337818,1.038503,1.856182,-0.462404,-0.612423,-0.759847,-0.629157,-0.012881,0.344711,-0.707889,-0.085616,0.0,0.0,0.0,1.0,0.0


In [ ]:
label.head()

,median_house_value
0,452600.0
1,358500.0
2,352100.0
3,341300.0
4,342200.0


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X.values,label.values,test_size=0.2,random_state=42)

In [ ]:
model.predict(x_test.iloc[0].ravel().reshape(1,-1))

array([178785.10865057])

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

C- It is a hypermeter in SVM to control error. High C mean it may get higher error and low C mean it can get lower error

But we need to bear in mind is if the C set in very low level, it may cause overfitting of the model.

Gamma is a hypermeter which is set before the training model and used to give curvature weight of the decision boundary

*** If we use kernel = 'linear', we don't need gamma ***

In [ ]:
model=SVR()
parameter_tuning={'kernel':['linear','rbf'],'C':[0.01,0.1,1,10],'gamma':[0.01,0.1,1,10]}

GridSearch=GridSearchCV(model,param_grid=parameter_tuning,scoring='neg_root_mean_squared_error',verbose=2)
GridSearch.fit(x_train,y_train.ravel())

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] C=0.01, gamma=0.01, kernel=linear ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ C=0.01, gamma=0.01, kernel=linear, total= 1.4min
[CV] C=0.01, gamma=0.01, kernel=linear ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV] ................ C=0.01, gamma=0.01, kernel=linear, total= 1.4min
[CV] C=0.01, gamma=0.01, kernel=linear ...............................
[CV] ................ C=0.01, gamma=0.01, kernel=linear, total=  50.4s
[CV] C=0.01, gamma=0.01, kernel=linear ...............................
[CV] ................ C=0.01, gamma=0.01, kernel=linear, total= 1.1min
[CV] C=0.01, gamma=0.01, kernel=linear ...............................
[CV] ................ C=0.01, gamma=0.01, kernel=linear, total= 1.1min
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV] ................... C=0.01, gamma=0.01, kernel=rbf, total= 2.1min
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV] ................... C=0.01, gamma=0.01, kernel=rbf, total=  49.8s
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV] ................... C=0.01, gamma=0.01, kernel=rbf, total= 1.4min
[CV] C=0.01, gamma=0.01, kernel=rbf ..................................
[CV] .

[CV] ..................... C=0.1, gamma=0.1, kernel=rbf, total= 1.5min
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total= 1.1min
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total= 1.2min
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total= 1.3min
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total=  57.3s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] .................... C=0.1, gamma=1, kernel=linear, total= 1.4min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total= 1.6min
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .

[CV] ........................ C=1, gamma=10, kernel=rbf, total= 2.4min
[CV] C=1, gamma=10, kernel=rbf .......................................
[CV] ........................ C=1, gamma=10, kernel=rbf, total= 2.2min
[CV] C=1, gamma=10, kernel=rbf .......................................
[CV] ........................ C=1, gamma=10, kernel=rbf, total= 2.0min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 1.2min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 1.3min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total=  48.8s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .................. C=10, gamma=0.01, kernel=linear, total= 1.0min
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed: 215.9min finished


GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['linear', 'rbf']},
             scoring='neg_root_mean_squared_error', verbose=2)

In [ ]:
prediction=GridSearch.predict(x_test)

In [ ]:
GridSearch.best_estimator_

SVR(C=10, gamma=0.01, kernel='linear')

In [ ]:
GridSearch.best_score_

-84238.69348689032

In [ ]:
cv_results = GridSearch.cv_results_
for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    # root mean square error with best estimators
    print(np.sqrt(-mean_score), params)

344.5337786369887 {'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}
344.64784524883567 {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
344.5337786369887 {'C': 0.01, 'gamma': 0.1, 'kernel': 'linear'}
344.6450709981776 {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
344.5337786369887 {'C': 0.01, 'gamma': 1, 'kernel': 'linear'}
344.6491489296766 {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
344.5337786369887 {'C': 0.01, 'gamma': 10, 'kernel': 'linear'}
344.6496702593959 {'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}
343.6139381392193 {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}
344.631255469056 {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
343.6139381392193 {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
344.6034718079113 {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
343.6139381392193 {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
344.6443031620075 {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
343.6139381392193 {'C': 0.1, 'gamma': 10, 'kernel': 'linear'}
344.64959775702835 {'C': 0.1, 'gamma': 10, 'kernel': 'rbf'}
334.98138196035

2. Try to build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set. Hint: the KNeighborsClassifier works quite well for this task; you just need to find good hyperparameter values (try a grid search on the weights and n_neighbors hyperparameters).

In [ ]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
mnist=datasets.load_digits()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(np.array(mnist.data),mnist.target, test_size=0.3, random_state=42)

In [ ]:
KNN_model=KNeighborsClassifier()
para={'n_neighbors':[3,5,7,9],'weights':['uniform','distance']}
KNN_Grid=GridSearchCV(KNN_model,param_grid=para,verbose=2)
KNN_Grid.fit(X_train,Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] n_neighbors=3, weights=uniform ..................................
[CV] ................... n_neighbors=3, weights=uniform, total=   1.6s
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] ................... n_neighbors=3, weights=uniform, total=   0.4s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ................... n_neighbors=3, weights=uniform, total=   0.4s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ................... n_neighbors=3, weights=uniform, total=   0.3s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ................... n_neighbors=3, weights=uniform, total=   0.4s
[CV] n_neighbors=3, weights=distance .................................
[CV] .................. n_neighbors=3, weights=distance, total=   0.2s
[CV] n_neighbors=3, weights=distance .................................
[CV] .................. n_neighbors=3, weights=distance, total=   0.3s
[CV] n_neighbors=3, weights=distance .................................
[CV] .................. n_neighbors=3, weights=distance, total=   0.2s
[CV] n_neighbors=3, weights=distance .................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   14.2s finished


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             verbose=2)

In [ ]:
knn_prediction=KNN_Grid.predict(X_test)

In [ ]:
KNN_Grid.best_estimator_

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [ ]:
class_report=classification_report(Y_test,knn_prediction)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       0.98      1.00      0.99        50
           2       1.00      1.00      1.00        47
           3       0.98      1.00      0.99        54
           4       0.98      1.00      0.99        60
           5       0.99      1.00      0.99        66
           6       1.00      1.00      1.00        53
           7       1.00      0.98      0.99        55
           8       0.98      0.98      0.98        43
           9       0.98      0.93      0.96        59

    accuracy                           0.99       540
   macro avg       0.99      0.99      0.99       540
weighted avg       0.99      0.99      0.99       540

